# This notebook contains background theory for the Compton Scattering Problem, and a solution using AI Hilbert.

You will need to have installed Julia, CPLEX (or Gurobi) to reproduce the code run here.

We ran everything using Julia version 1.7.3, and CPLEX

If you wish to use Gurobi, comment out the "using CPLEX" line and uncomment the "using Gurobi" line

In [1]:
using DynamicPolynomials
using JuMP
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using CPLEX
#using Gurobi

### Functions for generating monomials

We have two versions of these functions: one which only controls the overall degree, and one which also controls each variable's degree, in order to have finer grained control

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

mons_of_max_degree_and_unit_overall (generic function with 1 method)

In [3]:
@polyvar f1 lambda1 f2 lambda2 h me c cos E1 E2 Ee1 Ee2 p1 p2 pe2
u =[f1 lambda1 f2 lambda2 h me c cos E1 E2 Ee1 Ee2 p1 p2 pe2]

1×15 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 f1  lambda1  f2  lambda2  h  me  c  cos  E1  E2  Ee1  Ee2  p1  p2  pe2

In [4]:
# Including some axioms here
axioms2 =[
    E1+Ee1-E2-Ee2, # conservation of energy, E1 is initial photon energy, E2 final energy, Ee1/Ee2 is initial/final electron energy
    E1-h*f1,       # photon energy in terms of frequency (f1 is initial frequency, f2 is final frequency)
    E2-h*f2,
    p1*c - h*f1,   # photon momemntum in terms of frequency (p1 is initial photon momentum, p2 is final momentum)
    p2*c - h*f2,
    lambda1*f1-c,  # relationship between frequency and wavelength
    lambda2*f2-c,
    Ee1-me*c^2,    # energy of essentially stationary (relative to speed of light) electron
    Ee2^2-pe2^2*c^2-me^2*c^4, #relativistic energy-momentum relation applied to the electron after collision (after squaring); pe2 is final momentum
    pe2^2-p1^2-p2^2+2*p1*p2*cos, # conservation of momentum, after taking inner product of vector representation of conservation law
    ]

10-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 -Ee2 + Ee1 - E2 + E1
 E1 - f1h
 E2 - f2h
 cp1 - f1h
 cp2 - f2h
 -c + f1lambda1
 -c + f2lambda2
 Ee1 - mec²
 Ee2² - c²pe2² - me²c⁴
 pe2² - p2² - p1² + 2cosp1p2

In [5]:
### deg = 2
deg_overall=6
deg_overall_q=8 # degree of final polynomial law

deg_elementwise=[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] 
deg_elementwise_q=[2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2]

deg = 8
#maxD = 1

candidate_mons = [
    mons_of_max_degree_and_unit_overall(u, deg, deg_overall, deg_elementwise)
    for ai=axioms2
]
@show size.(candidate_mons)
#@show candidate_mons

# If you want to use Gurobi instead of CPLEX, you need to comment out the next line, and uncomment out the subsequent line
model = Model(CPLEX.Optimizer)
# model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "Presolve", 0)
# set_optimizer_attribute(model, "OptimalityTol", 1e-9)
# set_optimizer_attribute(model, "FeasibilityTol", 1e-9)
# set_optimizer_attribute(model, "IntFeasTol", 1e-9)
# set_optimizer_attribute(model, "MIPGap", 1e-9)
# set_optimizer_attribute(model, "Quad", 1)

# we are assuming only the initial and final wavelengths of light, the cosine of the angle of scatering are measurable.
# Also, we assume h, me, c (Planck's constant, rest mass of electron, speed of light) are known and thus can appear in final expression
mons_q = mons_of_max_degree_and_unit_overall([lambda1 lambda2 cos h me c] , deg, deg_overall_q, deg_elementwise_q)
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")
abs_coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="abq")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
#@show q

coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)

αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]
#@show αs

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms2));
eqs = coefficients(residual)
#@show residual
#@show eqs


# Ensure that the sum of the coefficients on the terms involving p isn't zero, in order that p is part of expression
@constraint model sum(coeff_q[degree.(mons_q, lambda1).>0]) == 1.0

@constraint model eqs .== 0
@constraint model abs_coeff_q.>=coeff_q
@constraint model abs_coeff_q.>=-coeff_q
#@variable(model, t[i=1:maxD]>=0.0)
#@constraint(model, imposeabs1[i=1:maxD], t[i]>=q(data_var[i,:]))
#@constraint(model, imposeabs2[i=1:maxD], t[i]>=-q(data_var[i,:]))
#@constraint(model, q(data_var[10,:]) == 0)
@objective model Min sum(abs_coeff_q) # Reduce number of non-zero coefficients via Lasso trick
#@objective model Min 100*sum(t) # Reduce number of non-zero coefficients via Lasso trick

optimize!(model)
@show termination_status(model)
#@show q
#@show value.(coeff_q)
#@show value.(abs_coeff_q)


value_poly2 = p -> sum(value.(coefficients(p)).* monomials(p))
@show value_poly2(q)
@show value_αs=value_poly2.(αs)

size.(candidate_mons) = [(54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,)]
size.(coeff_αs) = [(54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,), (54264,)]
CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Parallel mode: deterministic, using up to 8 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 6 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 420157 rows and 370276 columns.
Aggregator did 175781 substitutions.
Reduced LP has 1977 rows, 2589 columns, and 7208 nonzeros.
Presolve time = 1.17 sec. (349.78 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual infeasibility =            47.000000
Iteration:   171   Dual infeasibility =            33.999990
Iteration:   347   Dual infeasibility =            15.99999

10-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 -0.25lambda2hc + 0.25lambda1hc - 0.25lambda1lambda2Ee2 - 0.25lambda1lambda2Ee1 - 0.25lambda2hccos - 0.25lambda1lambda2cosEe2 - 0.25lambda1lambda2cosEe1 + 0.25f1lambda1²lambda2cosp2
 0.25lambda2hc - 0.25lambda1hc + 0.25lambda1lambda2Ee2 - 0.25lambda1lambda2Ee1 + 0.5lambda1²Ee1 + 0.25lambda2hccos + 0.25lambda1lambda2cosEe2 + 0.25lambda1lambda2cosE2 + 0.5lambda1lambda2mec² - 0.5lambda1²mec² + 0.25lambda1lambda2mec²cos - 0.5f1lambda1²lambda2cosp2
 -0.25lambda2hc - 0.25lambda1lambda2Ee2 - 0.25lambda1lambda2Ee1 - 0.25lambda1lambda2cosEe2 - 0.25lambda1lambda2cosE1 + 0.5lambda2hccos² + 0.25lambda1f2lambda2²p2 - 0.5lambda1lambda2ccos²p1 - 0.25lambda1lambda2mec²cos + 0.25lambda1f2lambda2²cosp2
 -0.25lambda2hc - 0.25lambda2hccos + 0.5lambda1lambda2cos²E2 - 0.25f1lambda1²lambda2p1 + 0.5f1lambda1²lambda2cosp2 - 0.25f1lambda1²lambda2cosp1
 -0.25lambda1lambda2E2 + 0.25lambda1la